In [3]:
import re
import pandas as pd
import numpy as np

In [4]:
inp = """Q: q0, q1, q2, q3, q4,
         E: a, b, c,
         F: q4,
         D: {
         q0, a, q1,
         q1, b, q2,
         q2, c, q0,
         q1, b, q3,
         q3, a, q4,
         q3, b, q0
         }
         """

In [56]:
class FiniteAutomaton():
    def __init__(self):
        self.Q = []
        self.E = []
        self.F = []
        self.D = []
        self.nfa = {}
        self.dfa = {}
        self.nfa_table = []
        self.dfa_table = []
        
    def read_input(self, inp):
#       some clever regex stuff to split the automata into a list of lists of symbols  
        lis = re.split(':|\n', inp)[: -1]
        automata = []
        for item in lis:
            if (len([symbol for symbol in re.split('\W+', item) if symbol != '']) != 0):
                automata += [[symbol for symbol in re.split('\W+', item) if symbol != '']]
#         print(automata)
        
        for i in range(len(automata)):
            if automata[i][0] == "Q":
                self.Q = automata[i + 1]
            if automata[i][0] == "E":
                self.E = automata[i + 1]
            if automata[i][0] == "F":
                self.F = automata[i + 1]
            if automata[i][0] == "D":
                self.D = [automata[j] for j in range(i + 1, i + 1 + len(automata) - 7)]
                
        for item in self.D:
            flag = 1
            if item[0] not in self.nfa.keys():
                self.nfa[item[0]] = [[item[1], [item[2]]]]
            else:
                for i in range(len(self.nfa[item[0]])):
                    if self.nfa[item[0]][i][0] == item[1]:
                        self.nfa[item[0]][i][1] += [item[2]]
                        flag = 0
                if flag == 1:
                    self.nfa[item[0]] += [[item[1], [item[2]]]]
                
    
    def print_nfa(self):
        tabel = np.empty((len(self.Q), len(self.E)))
        tabel[:] = np.nan
        tabel = pd.DataFrame(tabel, index=self.Q, columns=self.E)
        tabel.iloc[:] = '-'
#         print(tabel.loc["q0","a"])
        for key in self.nfa.keys():
            for pair in self.nfa[key]:
                tabel.loc[key, pair[0]] = "".join([str(item) for item in pair[1]])
        self.nfa_table = tabel
        print(self.nfa_table)
        
#     def to_dfa(self):
#         s = ["q0"]
#         self.dfa["q0"] = self.nfa["q0"]
#         while True:
#             for node in self.nfa_table.loc[s, :]:
#                 s = []

In [57]:
automaton = FiniteAutomaton()

In [58]:
automaton.read_input(inp)

In [59]:
automaton.print_nfa()

     a     b   c
q0  q1     -   -
q1   -  q2q3   -
q2   -     -  q0
q3  q4    q0   -
q4   -     -   -
